In [1]:
import re
import spacy
import numpy as np
import pandas as pd
import json
from tqdm import tqdm
import pickle
from sklearn.model_selection import KFold
from evaluate import run_eval
import itertools

In [7]:
def add_tag(labels, tags=["BIE", "S", "O"]):
    # 设置序列标注的标签体系
    # 1: BIE,S,O 
    # 2: BME,S,O
    outs=[]
    if len(labels) == 1:
        # 只有一个数
        finalTag = tags[1] + "_" + labels[-1] if labels[-1] != tags[2] else labels[-1]
        outs.append(finalTag)
    else:
        # 位置标签： B开始
        preTag = tags[0][0]
        curTag = tags[0][0]
        # 前一个类别
        preClass = labels[0]
        for i in range(1, len(labels)):
            # 当前类别
            curClass = labels[i]
            if curClass != preClass:
                if preTag == tags[0][0]:
                    preTag = tags[1]
                else:
                    preTag = tags[0][-1]
                curTag = tags[0][0]
            else:
                curTag = tags[0][1]
            # the former
            finalTag = preTag + "_" + labels[i-1] if labels[i-1] != tags[2] else labels[i-1]
            outs.append(finalTag)
            # last one
            if i >= len(labels) - 1:
                if curTag == tags[0][0]:
                    curTag = tags[1]
                else:
                    curTag = tags[0][-1]
                finalTag = curTag + "_" + labels[i] if labels[i] != tags[2] else labels[i]
                outs.append(finalTag)
                break
            preTag = curTag
            preClass = labels[i]
#     print("labels={}\nouts={}".format(labels, outs))
    return outs

def split_sent(sentIds, tokens, feats, labels):
    tokenid2sentid={}
    for i, t in enumerate(sentIds):
        for j in range(t[0], t[1]):
            tokenid2sentid[j]=i
    newTokens, newLabels, newFeats=[[]],[[]],[[]]
    assert len(tokenid2sentid) == len(tokens)
    preId=tokenid2sentid[0]
    for idt,(t,f,lab) in enumerate(zip(tokens, feats, labels)):
        if tokenid2sentid[idt] != preId:
            newTokens.append([])
            newFeats.append([])
            newLabels.append([])
        newTokens[-1].append(t)
        newFeats[-1].append(f)
        newLabels[-1].append(lab)
        preId=tokenid2sentid[idt]
#     print("{} newTokens={}\n{} newFeats={}\n{} newLabels={}\n".format(len(newTokens),newTokens, 
#                                                                       len(newFeats),newFeats, 
#                                                                       len(newLabels),newLabels))
    return newTokens, newFeats, newLabels
    
    
def tocrf(data, fout, hyphen="\t", labelName="LABEL"):
    outs=[]
    num=0
    for i in tqdm(data):
        # 标签转换
        labels=list(map(lambda x: str(labelName) if x==1 else "O", i[1]["label"]))
        # 增加BIES位置标签
        labels=add_tag(labels, tags=["BIE", "S", "O"])
#         print(i[1]["label"], "\n\n", labels)
        newTokens, newFeats, newLabels=split_sent(i[1]["sentIds"], i[1]["token"], i[1]["feat"], labels)
        for (tokenList, featList, labList) in zip(newTokens, newFeats, newLabels):
#             print(labList)
#             if list(set(labList))==["O"]:
#                 continue
            for (t,f,lab) in zip(tokenList, featList, labList):
                if t[2].strip() == "":
                    continue
                content=str(t[2]) + hyphen + hyphen.join(f[1:]) + hyphen + str(lab) + "\n"
                outs.append(content)
            outs.append(" \n")
            num+=1
    print("{} sentences".format(num))
    with open(fout, "w") as f:
        f.writelines(outs)

In [5]:
# 加载数据
basedir="/home/hebin/python_work/algorithm_nlp_toefl_writing_practiceAppraise/appraiser/src/modules/discourse/train/201_deterNoun/"

fout=f"{basedir}/sample2values.pkl"
with open(fout, "rb") as fw:
    sample2values=pickle.load(fw)

In [8]:
def gen_grid_paras(parameters):
    keys, pools=[],[]
    for k, v in parameters.items():
        keys.append(k)
        pools.append(tuple(v))
    result = [[]]
    for pool in pools:
        result = [x+[y] for x in result for y in pool]
    results={}    
    for idr, i in enumerate(result):
        results[idr]={k:v for k, v in zip(keys, i)}
    print(len(results), results)
    return results

In [9]:
parameters={"c": [10, 15, 25, 35, 40, 45, 50],
            "f": [1, 2],
           }
paras=gen_grid_paras(parameters)
# 拆分数据集
kf = KFold(n_splits=5, shuffle=True, random_state=42)
sv=list(sample2values.items())

results={}
for train_set, test_set in kf.split(sv):
    # 训练、测试数据文件生成
    trainDatas=[sv[i] for i in train_set]
    testDatas=[sv[i] for i in test_set]
    print("{} trainDatas, {} testDatas".format(len(trainDatas), len(testDatas)))
    with open(f"{basedir}/trainDatas.pkl", "wb") as fw:
        pickle.dump(trainDatas, fw)
    with open(f"{basedir}/testDatas.pkl", "wb") as fw:
        pickle.dump(testDatas, fw)

    tocrf(trainDatas, f"{basedir}/train-crf.data", hyphen="\t", labelName="LABEL")
    tocrf(testDatas, f"{basedir}/test-crf.data", hyphen="\t", labelName="LABEL")
    
    # 模型训练
    for p in paras.values():
        content=" ".join([f"-{k} {v}" for k,v in p.items()])
        cmd_train=f"crf_learn -p 32 {content} template train.data crfpp_model.bin\ncrf_test -m crfpp_model.bin test.data > test.result"
        with open("run.sh", "w") as fw2:
            fw2.write(cmd_train)
        !sh run.sh
        # 在测试数据集上评价结果
        f1=run_eval("./test.result", head=2, verbose=True)
        print("{}, f1={}".format(content, f1))
        if content not in results.keys():
            results[content]=[f1]
        else:
            results[content].append(f1)
    print("\n\n")

14 {0: {'c': 10, 'f': 1}, 1: {'c': 10, 'f': 2}, 2: {'c': 15, 'f': 1}, 3: {'c': 15, 'f': 2}, 4: {'c': 25, 'f': 1}, 5: {'c': 25, 'f': 2}, 6: {'c': 35, 'f': 1}, 7: {'c': 35, 'f': 2}, 8: {'c': 40, 'f': 1}, 9: {'c': 40, 'f': 2}, 10: {'c': 45, 'f': 1}, 11: {'c': 45, 'f': 2}, 12: {'c': 50, 'f': 1}, 13: {'c': 50, 'f': 2}}
3607 trainDatas, 902 testDatas


  0%|          | 0/902 [00:00<?, ?it/s]

52518 sentences


100%|██████████| 902/902 [00:00<00:00, 2843.91it/s]


13581 sentences
CRF++: Yet Another CRF Tool Kit
Copyright (C) 2005-2013 Taku Kudo, All rights reserved.

reading training data: 100.. 200.. 300.. 400.. 500.. 600.. 700.. 800.. 900.. 1000.. 1100.. 1200.. 1300.. 1400.. 1500.. 1600.. 1700.. 1800.. 1900.. 2000.. 2100.. 2200.. 2300.. 2400.. 2500.. 2600.. 2700.. 2800.. 2900.. 3000.. 3100.. 3200.. 3300.. 3400.. 3500.. 3600.. 3700.. 3800.. 3900.. 4000.. 4100.. 4200.. 4300.. 4400.. 4500.. 4600.. 4700.. 4800.. 4900.. 5000.. 5100.. 5200.. 5300.. 5400.. 5500.. 5600.. 5700.. 5800.. 5900.. 6000.. 6100.. 6200.. 6300.. 6400.. 6500.. 6600.. 6700.. 6800.. 6900.. 7000.. 7100.. 7200.. 7300.. 7400.. 7500.. 7600.. 7700.. 7800.. 7900.. 8000.. 8100.. 8200.. 8300.. 8400.. 8500.. 8600.. 8700.. 8800.. 8900.. 9000.. 9100.. 9200.. 9300.. 9400.. 9500.. 9600.. 9700.. 9800.. 9900.. 10000.. 10100.. 10200.. 10300.. 10400.. 10500.. 10600.. 10700.. 10800.. 10900.. 11000.. 11100.. 11200.. 11300.. 11400.. 11500.. 11600.. 11700.. 11800.. 11900.. 12000.. 12100.. 12200.. 1230

...

iter=7 terr=0.00485 serr=0.09318 act=4136435 obj=38320.50806 diff=0.97702
iter=8 terr=0.48804 serr=0.99477 act=4136435 obj=18376880.64724 diff=478.55733
iter=9 terr=0.00485 serr=0.09318 act=4136435 obj=37487.42174 diff=0.99796
iter=10 terr=0.00485 serr=0.09318 act=4136435 obj=38393.83525 diff=0.02418
iter=11 terr=0.00485 serr=0.09318 act=4136435 obj=29070.30607 diff=0.24284
iter=12 terr=0.00485 serr=0.09318 act=4136435 obj=29254.22414 diff=0.00633
iter=13 terr=0.00485 serr=0.09318 act=4136435 obj=27475.08554 diff=0.06082
iter=14 terr=0.00485 serr=0.09318 act=4136435 obj=24957.37080 diff=0.09164
iter=15 terr=0.00494 serr=0.09473 act=4136435 obj=22068.73780 diff=0.11574
iter=16 terr=0.00492 serr=0.09399 act=4136435 obj=19658.04258 diff=0.10924
iter=17 terr=0.00472 serr=0.09021 act=4136435 obj=18324.20956 diff=0.06785
iter=18 terr=0.00474 serr=0.09002 act=4136435 obj=16529.11135 diff=0.09796
iter=19 terr=0.00595 serr=0.11288 act=4136435 obj=23142.85855 diff=0.40013
iter=20 terr=0.00461 se

iter=118 terr=0.00024 serr=0.00482 act=4136435 obj=1367.49106 diff=0.01689
iter=119 terr=0.00023 serr=0.00463 act=4136435 obj=1342.28993 diff=0.01843
iter=120 terr=0.00022 serr=0.00429 act=4136435 obj=1296.61622 diff=0.03403
iter=121 terr=0.00021 serr=0.00410 act=4136435 obj=1265.27408 diff=0.02417
iter=122 terr=0.00019 serr=0.00378 act=4136435 obj=1248.92085 diff=0.01292
iter=123 terr=0.00019 serr=0.00374 act=4136435 obj=1232.83526 diff=0.01288
iter=124 terr=0.00019 serr=0.00372 act=4136435 obj=1222.12481 diff=0.00869
iter=125 terr=0.00018 serr=0.00359 act=4136435 obj=1212.20766 diff=0.00811
iter=126 terr=0.00020 serr=0.00389 act=4136435 obj=1191.24398 diff=0.01729
iter=127 terr=0.00018 serr=0.00359 act=4136435 obj=1171.04225 diff=0.01696
iter=128 terr=0.00018 serr=0.00348 act=4136435 obj=1161.49459 diff=0.00815
iter=129 terr=0.00017 serr=0.00340 act=4136435 obj=1148.74232 diff=0.01098
iter=130 terr=0.00017 serr=0.00334 act=4136435 obj=1181.18586 diff=0.02824
iter=131 terr=0.00017 ser

iter=229 terr=0.00009 serr=0.00179 act=4136435 obj=795.53542 diff=0.00186
iter=230 terr=0.00009 serr=0.00178 act=4136435 obj=795.06317 diff=0.00059
iter=231 terr=0.00009 serr=0.00178 act=4136435 obj=793.79045 diff=0.00160
iter=232 terr=0.00009 serr=0.00178 act=4136435 obj=793.21682 diff=0.00072
iter=233 terr=0.00009 serr=0.00178 act=4136435 obj=792.66844 diff=0.00069
iter=234 terr=0.00009 serr=0.00178 act=4136435 obj=791.75842 diff=0.00115
iter=235 terr=0.00009 serr=0.00179 act=4136435 obj=794.89971 diff=0.00397
iter=236 terr=0.00009 serr=0.00178 act=4136435 obj=791.45632 diff=0.00433
iter=237 terr=0.00009 serr=0.00178 act=4136435 obj=790.87950 diff=0.00073
iter=238 terr=0.00009 serr=0.00178 act=4136435 obj=790.62084 diff=0.00033
iter=239 terr=0.00009 serr=0.00178 act=4136435 obj=789.99881 diff=0.00079
iter=240 terr=0.00009 serr=0.00176 act=4136435 obj=789.57239 diff=0.00054
iter=241 terr=0.00009 serr=0.00176 act=4136435 obj=789.05172 diff=0.00066
iter=242 terr=0.00009 serr=0.00176 act

iter=340 terr=0.00009 serr=0.00176 act=4136435 obj=739.08693 diff=0.00051
iter=341 terr=0.00009 serr=0.00179 act=4136435 obj=739.44032 diff=0.00048
iter=342 terr=0.00009 serr=0.00178 act=4136435 obj=738.90760 diff=0.00072
iter=343 terr=0.00009 serr=0.00178 act=4136435 obj=738.74927 diff=0.00021
iter=344 terr=0.00009 serr=0.00176 act=4136435 obj=738.30118 diff=0.00061
iter=345 terr=0.00009 serr=0.00176 act=4136435 obj=737.70293 diff=0.00081
iter=346 terr=0.00009 serr=0.00179 act=4136435 obj=737.31982 diff=0.00052
iter=347 terr=0.00009 serr=0.00179 act=4136435 obj=736.82568 diff=0.00067
iter=348 terr=0.00009 serr=0.00179 act=4136435 obj=736.63196 diff=0.00026
iter=349 terr=0.00009 serr=0.00179 act=4136435 obj=736.47190 diff=0.00022
iter=350 terr=0.00009 serr=0.00176 act=4136435 obj=736.53801 diff=0.00009
iter=351 terr=0.00009 serr=0.00176 act=4136435 obj=736.37025 diff=0.00023
iter=352 terr=0.00009 serr=0.00176 act=4136435 obj=736.26828 diff=0.00014
iter=353 terr=0.00009 serr=0.00178 act

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [10]:
fresults={k: {"f1": v, "mean_f1":np.mean(v)} for k,v in results.items()}
print(pd.DataFrame(fresults).T.sort_values("mean_f1", ascending=False))

                                                           f1   mean_f1
-c 45 -f 1  [0.637873754152824, 0.6335697399527187, 0.6277...  0.624208
-c 50 -f 1  [0.6381766381766382, 0.6313799621928167, 0.630...  0.624111
-c 35 -f 1  [0.6361904761904761, 0.6313298627543776, 0.629...  0.623763
-c 40 -f 1  [0.6361904761904761, 0.6274323682961557, 0.629...  0.623613
-c 25 -f 1  [0.6312260536398467, 0.6302800189843379, 0.629...   0.62258
-c 10 -f 2  [0.6292237442922374, 0.6305818673883626, 0.627...  0.621541
-c 15 -f 2  [0.6353366470854044, 0.6286484059272565, 0.625...  0.620652
-c 15 -f 1  [0.6282420749279539, 0.6264945002391201, 0.627...  0.619216
-c 25 -f 2  [0.6268390548372715, 0.6276785714285714, 0.623...  0.618119
-c 35 -f 2  [0.6251113089937667, 0.6262806236080178, 0.620...  0.617851
-c 10 -f 1  [0.6274131274131275, 0.6210221793635486, 0.627...  0.616748
-c 45 -f 2  [0.6288751107174491, 0.6236178681999115, 0.616...  0.616063
-c 50 -f 2  [0.627763041556145, 0.6229508196721311, 0.6177...  0